In [16]:
from bs4 import BeautifulSoup
 
from googletrans import Translator

In [17]:
# Reading data from the xml file
with open('cv3.txt', 'r', encoding='utf-8') as f:
    data = f.read()
 

In [18]:
translator = Translator()

In [19]:

bs_data = BeautifulSoup(data, 'xml')
 
b_unique = bs_data.find_all('ATIVIDADES-DE-ENSINO')
lista = []
for i in b_unique:
    r = i.find_all('ENSINO')
    instituicao = i.find_parent().get('NOME-INSTITUICAO')
    for e in r:
        periodo = e.get('MES-INICIO')+'/'+e.get('ANO-INICIO')+'--'+e.get('MES-FIM')+'/'+e.get('ANO-FIM')
        curso = e.get('NOME-CURSO')
        nivel = e.get('TIPO-ENSINO')

        
        tag = "\item["+periodo+"]"+  "{\\textbf{"+translator.translate(instituicao, dest='en').text+"}}\\\\"
        tag = tag+"\n"+"{Role:} Lecturer \\\\"
        
        
        if(nivel == "GRADUACAO"):

            grad ="{Level: } Undergrad.\\\\"

        else: 
            grad ="{Level: } Grad. (Masters')\\\\"
        
        tag = tag+'\n'+grad
        

        
        tag = tag+'\n'+"{Program: } "+translator.translate(curso, dest='en').text+ "\\\\" 

        tag = tag+'\n'+"{Courses:}\n\\begin{itemize}"
        
        for j in e:
            d = j.get_text()

            tag = tag+'\n'+"\\item[--]  "+   translator.translate(d, dest='en').text.capitalize()

        tag = tag+'\n'+"\\end{itemize}"
        
        lista.append(tag)

In [20]:
lista.reverse()
for i in lista:
    print(i)

\item[07/2019--12/2019]{\textbf{State University of Maranhão}}\\
{Role:} Lecturer \\
{Level: } Grad. (Masters')\\
{Program: } Graduate Program in Computer and Systems Engineering\\
{Courses:}
\begin{itemize}
\item[--]  Special topics in computational intelligence
\end{itemize}
\item[09/2021--01/2022]{\textbf{State University of Maranhão}}\\
{Role:} Lecturer \\
{Level: } Grad. (Masters')\\
{Program: } Graduate Program in Computer and Systems Engineering\\
{Courses:}
\begin{itemize}
\item[--]  Data science
\end{itemize}
\item[03/2021--08/2021]{\textbf{State University of Maranhão}}\\
{Role:} Lecturer \\
{Level: } Grad. (Masters')\\
{Program: } Graduate Program in Computer and Systems Engineering\\
{Courses:}
\begin{itemize}
\item[--]  Machine learning
\end{itemize}
\item[08/2017--12/2017]{\textbf{State University of Maranhão}}\\
{Role:} Lecturer \\
{Level: } Grad. (Masters')\\
{Program: } Graduate Program in Computer and Systems Engineering\\
{Courses:}
\begin{itemize}
\item[--]  Special